# 1. <lesmis.csv> PageRank Analysis

In [1]:
import csv, networkx as nx, operator

g = nx.Graph()
reader = csv.reader(open('lesmis.csv', 'r'), delimiter=',')
data = [row for row in reader]
nodes = set([row[0] for row in data])
edges = [(row[0], row[2]) for row in data]

initial_rank = 1/float(len(nodes))
g.add_nodes_from(nodes, rank=initial_rank)
g.add_edges_from(edges)

### 1.1 (lesmis.csv) for loop로 구현

In [2]:
V = float(len(g))
s = 0.85
ranks = dict()

for key, node in g.nodes(data=True):
    ranks[key] = node.get('rank') #이전 셀에서 지정한 initial values

In [3]:
for _ in range(30):
    for key, node in g.nodes(data=True):
        rank_sum = 0.0
        neighbors = g[key]
        for n in neighbors:
            if ranks[n] is not None:
                rank_sum += (1/float(len(list(g.neighbors(n)))))*ranks[n]
        ranks[key] = ((1-s)*(1/V)) + s*rank_sum
        
sorted_ranks = sorted(ranks.items(), key=operator.itemgetter(1), reverse=True)

for i in range(30):
    print(sorted_ranks[i])

('Valjean', 0.07543036061492989)
('Myriel', 0.042779334746132824)
('Gavroche', 0.03576745495576416)
('Marius', 0.030895055642239704)
('Javert', 0.030302836800758605)
('Thenardier', 0.027926610139938784)
('Fantine', 0.027022812219932153)
('Enjolras', 0.021882118164082885)
('Cosette', 0.020611280056444546)
('MmeThenardier', 0.019501209144906696)
('Bossuet', 0.01895960617700118)
('Courfeyrac', 0.018578512415135875)
('Eponine', 0.01779396919631372)
('Mabeuf', 0.017478083760756147)
('Bahorel', 0.017199955621077803)
('Joly', 0.017199938857277552)
('Babet', 0.01669189628398639)
('Gueulemer', 0.01669189299046511)
('Claquesous', 0.01656108323508992)
('MlleGillenormand', 0.016260247015400714)
('Feuilly', 0.01589219778024098)
('Combeferre', 0.015892182321540837)
('Tholomyes', 0.01564748398652512)
('Bamatabois', 0.015576309235859427)
('Montparnasse', 0.01517097321714718)
('Gillenormand', 0.014957511775186673)
('Grantaire', 0.014456718779784801)
('Prouvaire', 0.013145934664093225)
('Zephine', 0.012

### 1.2 (lesmis.csv) MapReduce로 구현

In [4]:
pages = {}

for page_id, node in g.nodes(data=True):
    neighbors = g.neighbors(page_id)
    neighbors_ids = [neighbor for neighbor in neighbors] #이웃의 id를 따로 neighbors_ids 리스트에 지정
    value = [initial_rank, neighbors_ids]
    pages[page_id] = value

In [5]:
def mapper(page_id, value):
    neighbors = [neighbor for neighbor in g.neighbors(page_id)]
    out_pagerank = value[0]/float(len(neighbors))
    outputs = [(page_id, value)]
    for neighbor in neighbors:
        outputs.append((neighbor, out_pagerank))
    return outputs

def reducer(page_id, values):
    output = []
    rank_sum = 0.0
    if page_id not in result:
        result[page_id] = output
    for value in values:
        if type(value)==list:
            output = value
        else:
            rank_sum += value
    rank_sum = (1-s)*(1/V) + s*rank_sum
    output.insert(0, rank_sum)
    return (page_id, output)

def map_reduce(pages, mapper, reducer):
    intermediate = []
    for (page_id, value) in pages.items():
        intermediate.extend(mapper(page_id, value))
    groups = {}
    for (page_id, values) in intermediate:
        if page_id not in groups:
            groups[page_id] = []
        groups[page_id].append(values)
    result = {}
    for (page_id, values) in groups.items():
        (new_key, new_value) = reducer(page_id, values)
        result[new_key] = new_value
    
    return result

In [6]:
result = {}
for i in range(30):
    result = map_reduce(pages, mapper, reducer)

ranks = {}
for key, value in result.items():
    ranks[key] = value[0]

sorted_ranks = sorted(ranks.items(), key=operator.itemgetter(1), reverse=True)

for i in range(min(30, len(ranks))):
    print(sorted_ranks[i])

('Valjean', 0.0754325705628629)
('Myriel', 0.042780771929496524)
('Gavroche', 0.03576581219329744)
('Marius', 0.030893881254583037)
('Javert', 0.030302722981753596)
('Thenardier', 0.027926133667705723)
('Fantine', 0.027022610455348414)
('Enjolras', 0.021880921916695686)
('Cosette', 0.020611120248805858)
('MmeThenardier', 0.01950097767532012)
('Bossuet', 0.01895845410681974)
('Courfeyrac', 0.018577277714293575)
('Eponine', 0.017793461559582942)
('Mabeuf', 0.01747706246804825)
('Bahorel', 0.017198762987040973)
('Joly', 0.017198762987040973)
('Gueulemer', 0.01669162427078174)
('Babet', 0.01669162427078174)
('Claquesous', 0.016560801738472047)
('MlleGillenormand', 0.01626009151352342)
('Feuilly', 0.015891111569163133)
('Combeferre', 0.015891111569163133)
('Tholomyes', 0.0156471513808634)
('Bamatabois', 0.015576755712852655)
('Montparnasse', 0.015170728113858563)
('Gillenormand', 0.014957350502284635)
('Grantaire', 0.014455713491591873)
('Prouvaire', 0.013145026830567117)
('Zephine', 0.0126

# 2. <dophins.csv> PageRank Analysis

In [7]:
import csv, networkx as nx, operator

g = nx.Graph()
reader = csv.reader(open('dolphins.csv', 'r'), delimiter=',')
data = [row for row in reader]
nodes = set([row[0] for row in data])
edges = [(row[0], row[2]) for row in data]

initial_rank = 1/float(len(nodes))
g.add_nodes_from(nodes, rank=initial_rank)
g.add_edges_from(edges)

### 2.1 (dolphins.csv) for loop로 구현

In [8]:
V = float(len(g))
s = 0.85
ranks = dict()

for key, node in g.nodes(data=True):
    ranks[key] = node.get('rank') #이전 셀에서 지정한 initial values

In [9]:
for _ in range(30):
    for key, node in g.nodes(data=True):
        rank_sum = 0.0
        neighbors = g[key]
        for n in neighbors:
            if ranks[n] is not None:
                rank_sum += (1/float(len(list(g.neighbors(n)))))*ranks[n]
        ranks[key] = ((1-s)*(1/V)) + s*rank_sum
        
sorted_ranks = sorted(ranks.items(), key=operator.itemgetter(1), reverse=True)

for i in range(30):
    print(sorted_ranks[i])

('Grin', 0.032144661074043256)
('Jet', 0.031728242874119315)
('Trigger', 0.03129949562749031)
('Web', 0.030095503912994072)
('SN4', 0.029875507501931024)
('Topless', 0.02951436073814428)
('Scabs', 0.02842318672995743)
('Patchback', 0.02645865564842151)
('Gallatin', 0.026156982883970194)
('Beescratch', 0.0246508294741647)
('Kringel', 0.02464103469926497)
('SN63', 0.023939337011051824)
('Feather', 0.023458564620806737)
('SN9', 0.021966479469402438)
('Stripes', 0.021691217550549015)
('Upbang', 0.021650964797697257)
('SN100', 0.02061348065572507)
('DN21', 0.020053713666974424)
('Haecksel', 0.01988316209821579)
('Jonah', 0.01939565578270089)
('TR99', 0.019232029370781188)
('SN96', 0.017618739596473858)
('TR77', 0.01733958987165038)
('Number1', 0.017130151143936772)
('Double', 0.017098373632707164)
('Beak', 0.01696546725521975)
('MN105', 0.016939067247185753)
('MN83', 0.016905831841905763)
('Hook', 0.016626897859909517)
('SN90', 0.016137645145904465)


### 2.2 (dolphins.csv) MapReduce로 구현

In [10]:
pages = {}

for page_id, node in g.nodes(data=True):
    neighbors = g.neighbors(page_id)
    neighbors_ids = [neighbor for neighbor in neighbors] #이웃의 id를 따로 neighbors_ids 리스트에 지정
    value = [initial_rank, neighbors_ids]
    pages[page_id] = value

In [11]:
def mapper(page_id, value):
    neighbors = [neighbor for neighbor in g.neighbors(page_id)]
    out_pagerank = value[0]/float(len(neighbors))
    outputs = [(page_id, value)]
    for neighbor in neighbors:
        outputs.append((neighbor, out_pagerank))
    return outputs

def reducer(page_id, values):
    output = []
    rank_sum = 0.0
    if page_id not in result:
        result[page_id] = output
    for value in values:
        if type(value)==list:
            output = value
        else:
            rank_sum += value
    rank_sum = (1-s)*(1/V) + s*rank_sum
    output.insert(0, rank_sum)
    return (page_id, output)

def map_reduce(pages, mapper, reducer):
    intermediate = []
    for (page_id, value) in pages.items():
        intermediate.extend(mapper(page_id, value))
    groups = {}
    for (page_id, values) in intermediate:
        if page_id not in groups:
            groups[page_id] = []
        groups[page_id].append(values)
    result = {}
    for (page_id, values) in groups.items():
        (new_key, new_value) = reducer(page_id, values)
        result[new_key] = new_value
    
    return result

In [12]:
result = {}
for i in range(30):
    result = map_reduce(pages, mapper, reducer)

ranks = {}
for key, value in result.items():
    ranks[key] = value[0]

sorted_ranks = sorted(ranks.items(), key=operator.itemgetter(1), reverse=True)

for i in range(min(30, len(ranks))):
    print(sorted_ranks[i])

('Grin', 0.032143493442158756)
('Jet', 0.03172987260913328)
('Trigger', 0.031298461173290625)
('Web', 0.030097132110645918)
('SN4', 0.029874478226395832)
('Topless', 0.029513285559113984)
('Scabs', 0.02842221290110614)
('Patchback', 0.026457745524539136)
('Gallatin', 0.026158554472606777)
('Beescratch', 0.024651663857178114)
('Kringel', 0.024640340666397965)
('SN63', 0.023938558126031673)
('Feather', 0.023459983732054757)
('SN9', 0.021965930984060314)
('Stripes', 0.021690489047081075)
('Upbang', 0.021652012313247895)
('SN100', 0.02061326520484631)
('DN21', 0.020054875803600696)
('Haecksel', 0.01988252275398361)
('Jonah', 0.019394942163937418)
('TR99', 0.019231376478232906)
('SN96', 0.017618310074007503)
('TR77', 0.017339252660100602)
('Number1', 0.01713082750899772)
('Double', 0.017097908551891958)
('Beak', 0.01696498743988919)
('MN105', 0.016938455850072184)
('MN83', 0.016905222553468074)
('Hook', 0.01662631852870373)
('SN90', 0.016138504272230323)


# 분석 결과

### for loop 방식
- 그래프 내 하나의 노드에 대응하는 page rank값을 일대일로 구하고, 구한 page_rank를 리스트에 저장하는 방식을 취한다.  
  

### MapReduce 방식
1. mapper를 통해 각 노드의 이웃을 먼저  파악한 후 해당 노드가 이웃에게 부여할 rank값(out_pagerank = pagerank / neighbor의 수)을 구하여 이웃 노드에 분배한다. 그 결과 intermediate 리스트의 원소는 [(id, value), (이웃id, 분배 받은 rank), (이웃id, 분배 받은 rank), ...]형식을 갖게 된다.
2. groupbykey 작업을 통해 공통 키(노드 id)끼리 값을 묶어준다. 그 결과 groups 딕셔너리의 키에는 page_id가, 값에는 value 리스트와 분배받은 rank 값들이 들어온다. (ex. {page_1: value, rank, rank..., page_2: value, rank, rank..})
3. reducer를 통해 각 page의 page_rank를 계산한다. 그 결과 result 딕셔너리의 키는 page_id이고, 각 키의 첫번째 값에 page_rank가 저장된다.  

#### MapReduce는 대용량의 데이터를 일괄(병렬)처리 할 수 있으며, Pagerank의 계산을 Map과 Reduce라는 다소 단순한 함수를 통해 간단하게 구현할 수 있다.
#### for loop 방식과 MapReduce 방식의 결과로 도출되는 pagerank값은 아주 미묘한 차이가 있지만, 궁극적으로 pagerank '순위' 상의 차이는 없다.